In [13]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs
import math

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)

# Start the camera stream
pipeline.start(config)

# Create a pipeline profile to get the camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()

# Retrieve camera calibration parameters
fx = depth_intrinsics.fx
fy = depth_intrinsics.fy
cx = depth_intrinsics.ppx
cy = depth_intrinsics.ppy
k1, k2, p1, p2, k3 = depth_intrinsics.coeffs

# Load camera matrix and distortion coefficients
camera_matrix = np.array([[fx, 0, cx],
                          [0, fy, cy],
                          [0, 0, 1]])
dist_coeffs = np.array([k1, k2, p1, p2, k3])

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    # Convert the frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Perform object detection on the color image
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Convert 2D bounding box to 3D
                x_min, y_min, x_max, y_max = xyxy

                # Find the depth value at the center of the bounding box
                x_center = int((x_min + x_max) / 2)
                y_center = int((y_min + y_max) / 2)
                depth_value = depth_frame.get_distance(x_center, y_center)

                # Convert the depth value to 3D coordinates
                depth_point = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x_center, y_center], depth_value)
                x, y, z = depth_point

                # Create the object points for PnP
                object_points = np.array([[0, 0, 0],
                                          [0, 1, 0],
                                          [1, 1, 0],
                                          [1, 0, 0],
                                          [1, 1, 0]])

                # Create the image points for PnP
                image_points = np.array([[x_min, y_min],
                                         [x_min, y_max],
                                         [x_max, y_max],
                                         [x_max, y_min],
                                         [int((x_min + x_max) / 2), int((y_min + y_max) / 2)]])

                # Perform PnP
                success, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs)

                if success:
                    # Calculate tilt angle with respect to the camera
                    _, _, yaw = cv2.Rodrigues(rvec)[0]
                    tilt_angle = math.degrees(yaw)

                    # Draw bounding box and label on the color image
                    cv2.rectangle(color_image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
                    cv2.putText(color_image, f'Tilt Angle: {tilt_angle:.2f} degrees', (int(x_min), int(y_min) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the color image
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-1 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\solvepnp.cpp:840: error: (-215:Assertion failed) ( (npoints >= 4) || (npoints == 3 && flags == SOLVEPNP_ITERATIVE && useExtrinsicGuess) || (npoints >= 3 && flags == SOLVEPNP_SQPNP) ) && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'cv::solvePnPGeneric'


In [8]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt',force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt',source='local')
model.to(device).eval()

# Set camera parametersq
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    
    # Convert the frame to a numpy array
    frame = np.asanyarray(color_frame.get_data())

    # Perform object detection
    results = model(frame)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.7
            if conf > 0.5:
                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(frame, f'{cls}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Object Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-2 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-2 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [18]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs
import math

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)

# Start the camera stream
pipeline.start(config)

# Create a pipeline profile to get the camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()

# Retrieve camera calibration parameters
fx = depth_intrinsics.fx
fy = depth_intrinsics.fy
cx = depth_intrinsics.ppx
cy = depth_intrinsics.ppy
k1, k2, p1, p2, k3 = depth_intrinsics.coeffs

# Load camera matrix and distortion coefficients
camera_matrix = np.array([[fx, 0, cx],
                          [0, fy, cy],
                          [0, 0, 1]])
dist_coeffs = np.array([k1, k2, p1, p2, k3])

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    # Convert the frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Perform object detection on the color image
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Convert 2D bounding box to 3D
                x_min, y_min, x_max, y_max = xyxy

                # Create the object points for PnP
                object_points = np.array([[0, 0, 0],
                          [0, 1, 0],
                          [1, 1, 0],
                          [1, 0, 0]], dtype=np.float32)

                # Create the image points for PnP
                image_points = np.array([[x_min, y_min],
                                         [x_min, y_max],
                                         [x_max, y_max],
                                         [x_max, y_min]])

                # Perform PnP
                success, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs)

                if success:
                    # Calculate tilt angle with respect to the camera
                    _, _, yaw = cv2.Rodrigues(rvec)[0]
                    rotation_matrix, _ = cv2.Rodrigues(rvec)
                    tilt_vector = rotation_matrix[:, 1]  # Get the second column of the rotation matrix
                    tilt_angle = math.degrees(math.atan2(tilt_vector[2], tilt_vector[1]))


                    # Draw bounding box and label on the color image
                    cv2.rectangle(color_image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
                    cv2.putText(color_image, f'Tilt Angle: {tilt_angle:.2f} degrees', (int(x_min), int(y_min) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the color image
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-1 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [6]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Camera intrinsic parameters (example values)
fx = 421.0  # Focal length in x-axis
fy = 421.0  # Focal length in y-axis
cx = 427.0  # Principal point x-coordinate
cy = 236.0  # Principal point y-coordinate

# Camera extrinsic parameters (example values)
rotation_matrix = np.array([[   0.99997,  -0.007577,  -0.0027762],
                            [  0.0075674,     0.99997,  -0.0034583],
                            [  0.0028023,   0.0034372,     0.99999]])
translation_vector = np.array([   0.015133,-4.4939e-05,0.0001245])

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Perform object detection
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.7
            if conf > 0.5:
                # Draw bounding box and label on the frame
                cv2.rectangle(color_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(color_image, f'{conf}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                            (255, 0, 0), 2)

                # Get the top-left corner point of the bounding box
                x, y = int(xyxy[0]), int(xyxy[1])

                # Calculate the depth value at the top-left corner
                depth = depth_image[y, x] / 1000.0  # Convert from millimeters to meters

                # Calculate the camera coordinates (Xc, Yc, Zc)
                Xc = (x - cx) * depth / fx
                Yc = (y - cy) * depth / fy
                Zc = depth

                # Calculate the world coordinates (Xw, Yw, Zw) using camera extrinsics
                world_coordinates = np.dot(rotation_matrix, np.array([Xc, Yc, Zc])) + translation_vector
                Xw, Yw, Zw = world_coordinates

                # Draw three perpendicular lines from the top-left corner
                length = 50
                cv2.line(color_image, (x, y), (x, y - length), (0, 0, 255), 2)  # Red line (vertical)
                cv2.line(color_image, (x, y), (x + length, y), (0, 255, 0), 2)  # Green line (horizontal)
                cv2.line(color_image, (x, y), (x + length, y - length), (255, 0, 0), 2)  # Blue line (diagonal)

                # Display the world coordinates
                cv2.putText(color_image, f'X: {Xw:.2f}', (x + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(color_image, f'Y: {Yw:.2f}', (x + 10, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(color_image, f'Z: {Zw:.2f}', (x + 10, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [33]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model.to(device).eval()

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()

# Enable depth stream and retrieve camera calibration
config.enable_stream(rs.stream.depth, 0, 0, rs.format.z16, 30)
pipeline.start(config)
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()

# Set camera parameters
width, height = intrinsics.width, intrinsics.height
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Perform object detection
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Draw bounding box and label on the frame
                cv2.rectangle(color_image, (int(xy, 0), 2))
                cv2.putText(color_image, f'{conf}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                            (255, 0, 0), 2)

                # Get the top-left corner point of the bounding box
                x, y = int(xyxy[0]), int(xyxy[1])

                # Calculate the depth value at the top-left corner
                depth = depth_image[y, x] / depth_scale  # Convert to meters

                # Calculate the camera coordinates (Xc, Yc, Zc)
                Xc = (x - cx) * depth / fx
                Yc = (y - cy) * depth / fy
                Zc = depth

                # Draw three perpendicular lines from the top-left corner
                length = 50
                cv2.line(color_image, (x, y), (x, y - length), (0, 0, 255), 2)  # Red line (vertical)
                cv2.line(color_image, (x, y), (x + length, y), (0, 255, 0), 2)  # Green line (horizontal)
                cv2.line(color_image, (x, y), (x + length, y - length), (255, 0, 0), 2)  # Blue line (diagonal)

                # Display the camera coordinates
                cv2.putText(color_image, f'Xc: {Xc:.2f}', (x + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)
                cv2.putText(color_image, f'Yc: {Yc:.2f}', (x + 10, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)
                cv2.putText(color_image, f'Zc: {Zc:.2f}', (x + 10, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-1 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-1 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


RuntimeError: null pointer passed for argument "frame_ref"

Intrinsic Parameters:
- fx: 421.83587646484375
- fy: 421.83587646484375
- cx: 427.6553039550781
- cy: 236.85336303710938

Rotation Matrix:
[[    0.99997   -0.007577  -0.0027762]
 [  0.0075674     0.99997  -0.0034583]
 [  0.0028023   0.0034372     0.99999]]

Translation Vector:
[   0.015133 -4.4939e-05   0.0001245]


In [41]:
import pyrealsense2 as rs
import numpy as np

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 0, 0, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 0, 0, rs.format.bgr8, 30)
pipeline.start(config)

# Wait for a frameset
frameset = pipeline.wait_for_frames()
depth_frame = frameset.get_depth_frame()
color_frame = frameset.get_color_frame()

# Obtain the camera calibration intrinsics
depth_intrinsics = depth_frame.profile.as_video_stream_profile().intrinsics
color_intrinsics = color_frame.profile.as_video_stream_profile().intrinsics

# Get the intrinsic parameters
fx = depth_intrinsics.fx
fy = depth_intrinsics.fy
cx = depth_intrinsics.ppx
cy = depth_intrinsics.ppy

# Obtain the camera extrinsics
extrinsics = depth_frame.profile.get_extrinsics_to(color_frame.profile)

# Get the rotation matrix
rotation_matrix = np.reshape(extrinsics.rotation, (3, 3))

# Get the translation vector
translation_vector = np.array([extrinsics.translation[0], extrinsics.translation[1], extrinsics.translation[2]])

# Print the intrinsic parameters, rotation matrix, and translation vector
print("Intrinsic Parameters:")
print(f"- fx: {fx}")
print(f"- fy: {fy}")
print(f"- cx: {cx}")
print(f"- cy: {cy}")

print("\nRotation Matrix:")
print(rotation_matrix)

print("\nTranslation Vector:")
print(translation_vector)

# Stop the camera stream
pipeline.stop()


Intrinsic Parameters:
- fx: 421.83587646484375
- fy: 421.83587646484375
- cx: 427.6553039550781
- cy: 236.85336303710938

Rotation Matrix:
[[    0.99997   -0.007577  -0.0027762]
 [  0.0075674     0.99997  -0.0034583]
 [  0.0028023   0.0034372     0.99999]]

Translation Vector:
[   0.015133 -4.4939e-05   0.0001245]


In [5]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float32)

    # Normalize the depth values
    depth_image /= 1000.0  # Convert from millimeters to meters

    # Perform object detection
    results = model(depth_image)

    # Display the results in the color image
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Draw bounding box and label on the color image
                cv2.rectangle(color_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(color_image, f'{cls}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Draw bounding box on the depth image
                cv2.rectangle(depth_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)

    # Display the color image and depth image
    cv2.imshow('Color Image with Object Detection', color_image)
    cv2.imshow('Depth Image with Object Detection', depth_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate
# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float32)

    # Normalize the depth values
    depth_image /= 1000.0  # Convert from millimeters to meters

    # Perform object detection
    results = model(depth_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only calculate rotation and draw bounding box if confidence is greater than 0.5
            if conf > 0.5:
                # Calculate rotation with respect to x-axis
                center_x = (xyxy[0] + xyxy[2]) / 2
                depth_values = depth_image[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])].flatten() / 1000.0  # Convert from millimeters to meters
                avg_depth = np.mean(depth_values)
                rotation_x = np.arctan((center_x - width / 2) * avg_depth / fx) * (180 / np.pi)

                # Draw bounding box and rotation information on the depth image
                cv2.rectangle(depth_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(depth_image, f'Rotation: {rotation_x:.2f} degrees', (int(xyxy[0]), int(xyxy[1]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the depth image
    cv2.imshow('Depth Image with Object Detection', depth_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


NameError: name 'fx' is not defined

In [1]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float32)

    # Normalize the depth values
    depth_image /= 1000.0  # Convert from millimeters to meters

    # Perform object detection
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.7
            if conf > 0.5:
                # Draw bounding box and label on the frame
                cv2.rectangle(color_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(color_image, f'{conf}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                            (255, 0, 0), 2)

                # Get the center of the bounding box
                center_x = (xyxy[0] + xyxy[2]) / 2

                # Extract the depth values within the bounding box
                depth_values = depth_image[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])].flatten()

                # Calculate the average depth
                avg_depth = np.mean(depth_values)

                # Compute the x rotation
                rotation_x = np.arctan((center_x - width / 2) * avg_depth / fx) * (180 / np.pi)

                # Draw rotation information on the frame
                cv2.putText(color_image, f'Rotation X: {rotation_x:.2f}', (int(xyxy[0]), int(xyxy[1]) - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [8]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model = model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Camera intrinsic parameters (example values)
fx = 421.0  # Focal length in x-axis
fy = 421.0  # Focal length in y-axis
cx = 427.0  # Principal point x-coordinate
cy = 236.0  # Principal point y-coordinate

# Camera extrinsic parameters (example values)
rotation_matrix = np.array([[0.99997, -0.007577, -0.0027762],
                            [0.0075674, 0.99997, -0.0034583],
                            [0.0028023, 0.0034372, 0.99999]])
translation_vector = np.array([0.015133, -4.4939e-05, 0.0001245])

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Perform object detection
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only process if confidence is greater than 0.5
            if conf > 0.5:
                # Draw bounding box and label on the frame
                cv2.rectangle(color_image, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
                cv2.putText(color_image, f'{conf}', (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                            (255, 0, 0), 2)

                # Get the top-left corner point of the bounding box
                x, y = int(xyxy[0]), int(xyxy[1])

                # Calculate the depth value at the top-left corner
                depth = depth_image[y, x] / 1000.0  # Convert from millimeters to meters

                # Calculate the camera coordinates (Xc, Yc, Zc)
                Xc = (x - cx) * depth / fx
                Yc = (y - cy) * depth / fy
                Zc = depth

                # Calculate the world coordinates (Xw, Yw, Zw) using camera extrinsics
                world_coordinates = np.dot(rotation_matrix, np.array([Xc, Yc, Zc])) + translation_vector
                Xw, Yw, Zw = world_coordinates

                # Draw three perpendicular lines from the top-left corner
                length = 50
                cv2.line(color_image, (x, y), (x, y - length), (0, 0, 255), 2)  # Red line (vertical)
                cv2.line(color_image, (x, y), (x + length, y), (0, 255, 0), 2)  # Green line (horizontal)
                cv2.line(color_image, (x, y), (x + length, y - length), (255, 0, 0), 2)  # Blue line (diagonal)

                # Display the world coordinates
                cv2.putText(color_image, f'X: {Xw:.2f}', (x + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)
                cv2.putText(color_image, f'Y: {Yw:.2f}', (x + 10, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)
                cv2.putText(color_image, f'Z: {Zw:.2f}', (x + 10, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


cuboid

In [11]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')

model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    # Convert the frame to a numpy array
    frame = np.asanyarray(color_frame.get_data())

    # Perform object detection
    results = model(frame)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Draw cuboid around the object
                x1, y1, x2, y2 = map(int, xyxy)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Draw top face of cuboid
                cv2.line(frame, (x1, y1), (x2, y1), (255, 0, 0), 2)
                cv2.line(frame, (x2, y1), (x2, y2), (255, 0, 0), 2)
                cv2.line(frame, (x2, y2), (x1, y2), (255, 0, 0), 2)
                cv2.line(frame, (x1, y2), (x1, y1), (255, 0, 0), 2)

                # Draw vertical lines of cuboid
                cv2.line(frame, (x1, y1), (x1 + 20, y1 - 20), (255, 0, 0), 2)
                cv2.line(frame, (x2, y1), (x2 + 20, y1 - 20), (255, 0, 0), 2)
                cv2.line(frame, (x2, y2), (x2 + 20, y2 - 20), (255, 0, 0), 2)
                cv2.line(frame, (x1, y2), (x1 + 20, y2 - 20), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Object Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [1]:
import cv2
import torch
import numpy as np
import pyrealsense2 as rs

# Load the YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt', force_reload=True)
model = torch.hub.load('yolov5', 'custom', 'best.pt', source='local')
model.to(device).eval()

# Set camera parameters
width, height = 640, 480  # Set the desired frame size
fps = 30  # Set the desired frame rate

# Initialize the RealSense camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

# Start the camera stream
pipeline.start(config)

# Object detection loop
while True:
    # Wait for a new frame
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert the frames to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Perform object detection
    results = model(color_image)

    # Display the results
    for result in results.xyxy[0]:
        if result is not None:
            xyxy = result[:4].tolist()
            conf = result[4].item()
            cls = int(result[5].item())

            # Only draw bounding box and label if confidence is greater than 0.5
            if conf > 0.5:
                # Draw cuboid around the object
                x1, y1, x2, y2 = map(int, xyxy)
                cv2.rectangle(color_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Get the depth values for the corners of the bounding box
                depth_tl = depth_image[y1, x1]
                depth_tr = depth_image[y1, x2]
                depth_br = depth_image[y2, x2]
                depth_bl = depth_image[y2, x1]

                # Draw closed cuboid
                cv2.line(color_image, (x1, y1), (x1, y2), (255, 0, 0), 2)  # Front face
                cv2.line(color_image, (x1, y1), (x2, y1), (255, 0, 0), 2)
                cv2.line(color_image, (x2, y1), (x2, y2), (255, 0, 0), 2)
                cv2.line(color_image, (x2, y2), (x1, y2), (255, 0, 0), 2)
                cv2.line(color_image, (x1, y2), (x1, y1), (255, 0, 0), 2)

                cv2.line(color_image, (x1, y1), (x1, y1 - depth_tl), (255, 0, 0), 2)  # Front top edge
                cv2.line(color_image, (x2, y1), (x2, y1 - depth_tr), (255, 0, 0), 2)
                cv2.line(color_image, (x2, y2), (x2, y2 - depth_br), (255, 0, 0), 2)
                cv2.line(color_image, (x1, y2), (x1, y2 - depth_bl), (255, 0, 0), 2)

                cv2.line(color_image, (x1, y1 - depth_tl), (x2, y1 - depth_tr), (255, 0, 0), 2)  # Top face
                cv2.line(color_image, (x2, y1 - depth_tr), (x2, y2 - depth_br), (255, 0, 0), 2)
                cv2.line(color_image, (x2, y2 - depth_br), (x1, y2 - depth_bl), (255, 0, 0), 2)
                cv2.line(color_image, (x1, y2 - depth_bl), (x1, y1 - depth_tl), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Object Detection', color_image)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop the camera stream
pipeline.stop()

# Close all windows
cv2.destroyAllWindows()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Prabhu/.cache\torch\hub\master.zip
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2023-6-7 Python-3.10.4 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


IndexError: index 480 is out of bounds for axis 0 with size 480